# Estudio de la clasficacion en clusters de los usuarios

In [1]:
import pandas as pd
import numpy as np

In [2]:
users = pd.read_csv('data/Prado_users_clusters_OnePolarity.csv')
emotions = pd.read_csv('data/Prado_users_emotions_OnePolarity.csv')
artworks = pd.read_csv('data/Prado_artworks_wikidata.csv')

In [3]:
users['positive'] = emotions['positive'].apply(eval)
users['negative'] = emotions['negative'].apply(eval)
users['mixed'] = emotions['mixed'].apply(eval)

In [4]:
users

,userId,age,gender,country,cluster,medoid,positive,negative,mixed
0,1,55-64,male,Spain,2,-1.0,[57726f4cedc2cb3880ba07ed],"[577278d5edc2cb3880d94196, 57727c2eedc2cb3880e...","[5772700eedc2cb3880bc82ce, 57727015edc2cb3880b..."
1,2,25-34,female,Spain,0,-1.0,[],[57726f59edc2cb3880ba76b3],[]
2,3,18-24,female,Spain,4,-1.0,"[57726f4cedc2cb3880ba07ed, 57726f59edc2cb3880b...","[57726f76edc2cb3880baa425, 57726f78edc2cb3880b...","[57726fffedc2cb3880bc69d6, 57727015edc2cb3880b..."
3,4,18-24,female,Spain,1,-1.0,"[57726f4cedc2cb3880ba07ed, 57726f59edc2cb3880b...","[57726e45edc2cb3880b69781, 57727cbaedc2cb3880e...",[577278d5edc2cb3880d94196]
4,5,35-44,male,Spain,1,-1.0,"[57726e34edc2cb3880b62340, 57726f48edc2cb3880b...",[57727cbaedc2cb3880e561f7],[57727015edc2cb3880bceb08]
...,...,...,...,...,...,...,...,...,...
166,298,45-54,male,Italy,4,-1.0,[],[],[57727015edc2cb3880bceb08]
167,299,45-54,male,Spain,2,-1.0,[57726f59edc2cb3880ba76b3],[],[]
168,300,25-34,female,Spain,0,-1.0,[],[57727cbaedc2cb3880e561f7],[]
169,301,45-54,male,Spain,0,-1.0,[],[57727cbaedc2cb3880e561f7],[]


In [5]:
clusters = []
for c in users.cluster.unique():
  clusters.append(users[users['cluster'] == c])

In [6]:
atr = clusters[0].columns[1]
clusters[0][atr].mode()[0]

'18-24'

In [8]:
clusters[1].describe()

,userId,cluster,medoid
count,51.000000,51.0,51.000000
mean,137.764706,0.0,-0.921569
std,84.076296,0.0,0.560112
min,2.000000,0.0,-1.000000
25%,71.500000,0.0,-1.000000
50%,143.000000,0.0,-1.000000
75%,184.000000,0.0,-1.000000
max,306.000000,0.0,3.000000


In [9]:
import collections

In [13]:
users_promedio = pd.read_csv('data/Prado_users_emotions_scaled.csv')

for i, c in enumerate(clusters):
    print("-----------------------------------------------------------------------------------------------------")
    print('Cluster {}:'.format(c.cluster.iloc[0]))
    
    g_arr = []
    for g in c.gender.unique():
        g_arr.append(np.count_nonzero(c['gender'] == g))
        # g = 'male' if count_nonzero(c['gender'] == 'male') > np.count_nonzero(c['gender'] == 'female') else 'female'
    gender_max = c.gender.unique()[g_arr.index(max(g_arr))]
    
    age_arr = []
    for a  in c.age.unique():
        age_arr.append(np.count_nonzero(c['age'] == a))
    age_max = c.age.unique()[age_arr.index(max(age_arr))]
    
    country_arr = []
    for co in c.country.unique():
        country_arr.append(np.count_nonzero(c['country'] == co))
    country_max = c.country.unique()[country_arr.index(max(country_arr))]
    
    positive_arr = []
    for l in c.positive:
      for k in l:
        # title = artworks[artworks['ID'] == k]['Title'].iloc[0]
        # positive_arr.append(title)
        positive_arr.append(k)
        
    negative_arr = []
    for l in c.negative:
      for k in l:
        # title = artworks[artworks['ID'] == k]['Title'].iloc[0]
        # negative_arr.append(title)
        negative_arr.append(k)
        
    mixed_arr = []
    for l in c.mixed:
      for k in l:
        # title = artworks[artworks['ID'] == k]['Title'].iloc[0]
        # mixed_arr.append(title)
        mixed_arr.append(k)
    
    #######
    uProm = {"userId":'expl'+str(i),"age":2.933903194819822,"gender":0.0 if gender_max == 'female' else 2.021722956442927,"country":1.7874048008113603,"positive":positive_arr[:5],"negative":negative_arr[:5],"mixed":mixed_arr[:5]}
    users_promedio = users_promedio.append(uProm,ignore_index=True)
    #######
    
    print("numero de gente en cluster", i, sum(g_arr))
    print("Genero con mas aparicion en cluster", i,gender_max, "con un ", max(g_arr)/sum(g_arr)*100,"%")
    print("Edad con mas aparicion en cluster ", i, age_max , "con un ", max(age_arr)/sum(age_arr)*100,"%")
    print("Pais con mas aparicion en cluster ", i, country_max, "con un ", max(country_arr)/sum(country_arr)*100,"%")
    
    #-------------------------------------------------
    
    # sorted(zip(positive_arr, [c for c in collections.Counter(positive_arr).values()]), key=lambda x : x[1], reverse=True)
    pos = sorted(dict(collections.Counter(positive_arr)).items(), key=lambda item: item[1], reverse=True)
    
    pos_titles = [artworks[artworks['ID'] == id].Title.item() for id in [p[0] for p in pos]]
    print("\nCuadros más gustados: ", list(zip(pos_titles, pos)))
    
    pos_artists = [artworks[artworks['ID'] == id].Artist.item() for id in [p[0] for p in pos]]
    sorted_pos_artists = sorted(dict(collections.Counter(pos_artists)).items(), key=lambda item: item[1], reverse=True)
    print("\nArtistas más gustados: ", sorted_pos_artists)
    
    pos_categories = [artworks[artworks['ID'] == id].Category.item() for id in [p[0] for p in pos]]
    sorted_pos_categories = sorted(dict(collections.Counter(pos_categories)).items(), key=lambda item: item[1], reverse=True)
    print("\nCategorías más gustadas: ", sorted_pos_categories)
   
    #--------------------------------------------------
    
    neg = sorted(dict(collections.Counter(negative_arr)).items(), key=lambda item: item[1], reverse=True)
    
    neg_titles = [artworks[artworks['ID'] == id].Title.item() for id in [n[0] for n in neg]]
    print("\nCuadros menos gustados: ", list(zip(neg_titles, neg)))
    
    neg_artists = [artworks[artworks['ID'] == id].Artist.item() for id in [n[0] for n in neg]]
    sorted_neg_artists = sorted(dict(collections.Counter(neg_artists)).items(), key=lambda item: item[1], reverse=True)
    print("\nArtistas menos gustados: ", sorted_neg_artists)
    
    neg_categories = [artworks[artworks['ID'] == id].Category.item() for id in [n[0] for n in neg]]
    sorted_neg_categories = sorted(dict(collections.Counter(neg_categories)).items(), key=lambda item: item[1], reverse=True)
    print("\nCategorías menos gustadas: ", sorted_neg_categories)
    
    #---------------------------------------------------
    
#     mix = sorted(dict(collections.Counter(mixed_arr)).items(), key=lambda item: item[1], reverse=True)
    
#     mix_titles = [artworks[artworks['ID'] == id].Title.item() for id in [m[0] for m in mix]]
#     print("\nCuadros mixed: ", list(zip(mix_titles, mix)))
    
#     mix_artists = [artworks[artworks['ID'] == id].Artist.item() for id in [m[0] for m in mix]]
#     sorted_mix_artists = sorted(dict(collections.Counter(mix_artists)).items(), key=lambda item: item[1], reverse=True)
#     print("\nArtistas mixed: ", sorted_mix_artists)
    
#     mix_categories = [artworks[artworks['ID'] == id].Category.item() for id in [m[0] for m in mix]]
#     sorted_mix_categories = sorted(dict(collections.Counter(mix_categories)).items(), key=lambda item: item[1], reverse=True)
#     print("\nCategorías mixed: ", sorted_mix_categories)
  
  
    #---------------------------------------------------
    
    print("----------------------------------------------------------------------------------------------------\n")

-----------------------------------------------------------------------------------------------------
Cluster 2:
numero de gente en cluster 0 38
Genero con mas aparicion en cluster 0 male con un  89.47368421052632 %
Edad con mas aparicion en cluster  0 35-44 con un  18.421052631578945 %
Pais con mas aparicion en cluster  0 Spain con un  42.10526315789473 %

Cuadros más gustados:  [('The Straw Manikin', ('57726f59edc2cb3880ba76b3', 16)), ('Rest on the Flight into Egypt', ('57727c95edc2cb3880e4d1d4', 12)), ('St. Rufina of Seville', ('57727015edc2cb3880bceb08', 6)), ('The Surgeon', ('57727cbaedc2cb3880e561f7', 5)), ('The Virgin and the Child', ('57727c88edc2cb3880e4be65', 4)), ('Adoration by the Shepherds', ('57727d14edc2cb3880e6926e', 4)), ('Summer, or The Harvest', ('57726f56edc2cb3880ba7233', 4)), ('The Pearl of Brabant', ('57727c2eedc2cb3880e39405', 3)), ('Lucrezia di Baccio del Fede', ('57727d5cedc2cb3880e751a0', 3)), ('The Defense of Cadiz against the English', ('57727016edc2cb3880b

In [48]:
users_promedio.to_csv('data/Prado_users_individuo_explicador.csv', index=0)

In [49]:
users_promedio

,userId,age,gender,country,positive,negative,mixed
0,1,4.107464,2.021723,1.787405,"['57726f4cedc2cb3880ba07ed', '57726f4cedc2cb38...","['577278d5edc2cb3880d94196', '57727015edc2cb38...","['5772700eedc2cb3880bc82ce', '5772700eedc2cb38..."
1,2,2.347123,0.000000,1.787405,[],['57726f59edc2cb3880ba76b3'],"['57726f59edc2cb3880ba76b3', '57726f59edc2cb38..."
2,3,1.760342,0.000000,1.787405,"['577278d5edc2cb3880d94196', '5772752eedc2cb38...","['5772700eedc2cb3880bc82ce', '577278d5edc2cb38...",[]
3,4,1.760342,0.000000,1.787405,"['577278d5edc2cb3880d94196', '57726f59edc2cb38...","['57726e45edc2cb3880b69781', '577278d5edc2cb38...",[]
4,5,2.933903,2.021723,1.787405,"['57726e34edc2cb3880b62340', '57726e34edc2cb38...","['57727cbaedc2cb3880e561f7', '57727cbaedc2cb38...","['57727cbaedc2cb3880e561f7', '57727015edc2cb38..."
...,...,...,...,...,...,...,...
168,300,2.347123,0.000000,1.787405,"['57727cbaedc2cb3880e561f7', '57727cbaedc2cb38...","['57727cbaedc2cb3880e561f7', '57727cbaedc2cb38...",[]
169,301,3.520684,2.021723,1.787405,[],['57727cbaedc2cb3880e561f7'],"['57727cbaedc2cb3880e561f7', '57727cbaedc2cb38..."
170,306,4.107464,2.021723,1.787405,"['57726f59edc2cb3880ba76b3', '57726f59edc2cb38...","['57726f59edc2cb3880ba76b3', '57726f59edc2cb38...","['57726f59edc2cb3880ba76b3', '57726f59edc2cb38..."
171,expl0,2.933903,2.021723,1.787405,"[57726f4cedc2cb3880ba07ed, 57726f4cedc2cb3880b...","[577278d5edc2cb3880d94196, 57727015edc2cb3880b...","[5772700eedc2cb3880bc82ce, 5772700eedc2cb3880b..."


In [31]:
users[users['medoid'] != -1]

,userId,age,gender,country,cluster,medoid,positive,negative,mixed
129,175,35-44,male,Japan,0,1.0,[],"[57726e37edc2cb3880b628ee, 57726e37edc2cb3880b...","[57726e37edc2cb3880b628ee, 57726e37edc2cb3880b..."
140,199,25-34,female,Japan,1,0.0,"[57726e34edc2cb3880b62340, 57726e34edc2cb3880b...",[],[57726e34edc2cb3880b62340]


In [32]:
users_promedio.append(users[users['medoid'] != -1]).sort_values(by=['cluster'])

,userId,age,gender,country,positive,negative,mixed,cluster,medoid
129,175,35-44,male,Japan,[],"[57726e37edc2cb3880b628ee, 57726e37edc2cb3880b...","[57726e37edc2cb3880b628ee, 57726e37edc2cb3880b...",0.0,1.0
140,199,25-34,female,Japan,"[57726e34edc2cb3880b62340, 57726e34edc2cb3880b...",[],[57726e34edc2cb3880b62340],1.0,0.0
0,1,55-64,male,Spain,"['57726f4cedc2cb3880ba07ed', '57726f4cedc2cb38...","['577278d5edc2cb3880d94196', '57727015edc2cb38...","['5772700eedc2cb3880bc82ce', '5772700eedc2cb38...",NaN,NaN
1,2,25-34,female,Spain,[],['57726f59edc2cb3880ba76b3'],"['57726f59edc2cb3880ba76b3', '57726f59edc2cb38...",NaN,NaN
2,3,18-24,female,Spain,"['577278d5edc2cb3880d94196', '5772752eedc2cb38...","['5772700eedc2cb3880bc82ce', '577278d5edc2cb38...",[],NaN,NaN
...,...,...,...,...,...,...,...,...,...
168,300,25-34,female,Spain,"['57727cbaedc2cb3880e561f7', '57727cbaedc2cb38...","['57727cbaedc2cb3880e561f7', '57727cbaedc2cb38...",[],NaN,NaN
169,301,45-54,male,Spain,[],['57727cbaedc2cb3880e561f7'],"['57727cbaedc2cb3880e561f7', '57727cbaedc2cb38...",NaN,NaN
170,306,55-64,male,Spain,"['57726f59edc2cb3880ba76b3', '57726f59edc2cb38...","['57726f59edc2cb3880ba76b3', '57726f59edc2cb38...","['57726f59edc2cb3880ba76b3', '57726f59edc2cb38...",NaN,NaN
171,expl0,35-44,male,Spain,"[57726f4cedc2cb3880ba07ed, 57726f4cedc2cb3880b...","[577278d5edc2cb3880d94196, 57727015edc2cb3880b...","[5772700eedc2cb3880bc82ce, 5772700eedc2cb3880b...",NaN,NaN
